In [2]:
import re, os
import unicodedata
import json

import pandas as pd
import numpy as np
import seaborn as sns
import seaborn as sns
import matplotlib.pyplot as plt

import nltk
import nltk.sentiment
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from time import strftime

import wrangle
import acquire
from prepare import basic_clean, tokenize, lemmatize, stem, remove_stopwords, prep_string_data#, split_data

import re
from sklearn.feature_extraction.text import TfidfVectorizer

from wordcloud import WordCloud
from matplotlib import pyplot as plt
import seaborn as sns

import nltk

from scipy import stats

import warnings
warnings.filterwarnings("ignore")

---

## Acquire Data

Our data was acquired from kaggle.com by dowloading it directly from the [Interview: NPR Media Dialog Transcripts kaggle dataset](https://www.kaggle.com/shuyangli94/interview-npr-media-dialog-transcripts/code) website.

- After downloading the relevant .csv files, our working dataset comes from two different sources (`utterances.csv` and `episodes.csv`) combined into one. 
- Our acquire script also did the following:
 - Combined dataset was put together via inner join.
 - Unnecessary and/or redundant columns were dropped or renamed.  
 - The string columns were all converted to lowercase.
 - Finally, an additional column, is_host, was created to identify whether the speaker for a specific row was an NPR host or not.

In [10]:
joined_df = acquire.get_df()

---